In [17]:
import collections
import math
import random
import sys
import time
import os
import numpy as np
import torch
from torch import nn
import torch.utils.data as Data
import string
import mglearn

import pandas as pd
import gzip

sys.path.append("..") 
print(torch.__version__)

C:\Users\cnyin\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
C:\Users\cnyin\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


1.0.0


In [3]:
# novel "the Hunger Games"
with open('the_hunger_games.txt', 'r') as f:
    lines_hg = f.readlines()
    
# novel "Catching Fire"
with open('catching_fire.txt', 'r') as g:
    lines_cf = g.readlines()

In [35]:
imdb = pd.read_csv('imdb_master2.csv')
imdb_text = imdb['review'].tolist()

In [4]:
def clean_novel(lines):
    import re
    import string
    txt = ''.join(lines)
    sentences = [x for x in map(str.strip, re.split(',|\.|\n|\?|;|!', txt)) if x]
    pre_words = [x.translate(str.maketrans('', '', string.punctuation)).lower() for x in sentences]
    
    # only select sentences that have more than 3 words
    words = list(filter(lambda x: len(x.split()) > 3, pre_words))
    return(words)

In [41]:
imdb = clean_novel(imdb_text)[:50000]
len(imdb)

50000

In [42]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(stop_words="english", max_df=100)

X = vect.fit_transform(imdb)
print(X.shape)

(50000, 27809)


In [47]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=10, learning_method="batch",
                               max_iter=25, random_state=0)

document_topics = lda.fit_transform(X)

In [48]:
lda.components_.shape

(10, 27809)

In [49]:
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vect.get_feature_names())

In [55]:
mglearn.tools.print_topics(topics=range(10), feature_names=feature_names,
                          sorting=sorting, topics_per_chunk=8, n_words=20)

topic 0       topic 1       topic 2       topic 3       topic 4       topic 5       topic 6       topic 7       
--------      --------      --------      --------      --------      --------      --------      --------      
overall       type          drama         review        stories       talk          havent        whats         
footage       dumb          child         fails         turns         definitely    gun           reviews       
buy           unless        happens       cave          bother        rent          predictable   lots          
cat           god           roles         van           plain         stay          enjoyed       taken         
ride          unfortunately youve         box           check         sit           forced        pathetic      
editing       sequel        die           voice         garbage       nearly        wait          chance        
live          suspense      flat          ends          zombies       wasted        documentary 